In [1]:
import json
from gensim.models import Word2Vec
import re

C:\Users\Axel\Anaconda2\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def preprocess(article):
    article = article.encode('utf-8')
    article = article.lower()
    regex = re.compile('[^a-z\s]')
    article = regex.sub('', article)
    return article.split(" ")

In [3]:
path = r"C:\Users\Axel\github\thesis\Data\PoliFLW Data\kamerstukken_topics_first.json"

with open(path) as data_file:    
    data = json.load(data_file)
    
categories = []
content = []

for article in data:
    category = article["category"].split("|")
    categories.append(category[0])
    content.append(preprocess(article["content"]))
    
path = r"C:\Users\Axel\github\thesis\Data\PoliFLW Data\kamerstukken_topics_second.json"

with open(path) as data_file:    
    testData = json.load(data_file)
    
for article in testData:
    category = article["category"].split("|")
    categories.append(category[0])
    content.append(preprocess(article["content"]))

In [4]:
model = Word2Vec(content, size=100, window=5, min_count=5)

In [5]:
model.wv.most_similar(positive=['frankrijk'])
#print(model.wv.vocab)

[('denemarken', 0.8561245203018188),
 ('luxemburg', 0.8456012010574341),
 ('duitsland', 0.8286179304122925),
 ('belgi', 0.8060805201530457),
 ('zwitserland', 0.7735120058059692),
 ('oostenrijk', 0.7636076807975769),
 ('ierland', 0.7597377300262451),
 ('spanje', 0.7511701583862305),
 ('zweden', 0.7476193904876709),
 ('itali', 0.7416862845420837)]